In [ ]:
!pip install xmltodict
!pip install mean_average_precision

In [ ]:
import torch
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import functional as F
from torch.utils.data import DataLoader, Dataset
import numpy as np
import os
import time
from tqdm.notebook import tqdm
import random
import os
import cv2
import numpy as np
import torch
import xmltodict
from torch.utils.data import Dataset
import torchvision
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models import efficientnet_b3, EfficientNet_B3_Weights
from torchvision.transforms.functional import normalize

In [ ]:
plt.rcParams.update({'font.size': 14})  # 14pt is good for thesis plots


In [ ]:
import cv2
class BoatDetectionDataset(Dataset):
    def __init__(self, image_dir, annotation_dir, image_list=None, transform=None):
        self.image_dir = image_dir
        self.annotation_dir = annotation_dir
        self.transform = transform

        if image_list is not None:
            self.image_files = image_list  # use the provided list (train/val split)
        else:
            self.image_files = sorted([file for file in os.listdir(image_dir) if file.lower().endswith((".jpg", ".jpeg", ".png"))]

)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        annot_path = os.path.join(self.annotation_dir, os.path.splitext(self.image_files[idx])[0] + ".xml")

        with open(annot_path) as f:
            annot = xmltodict.parse(f.read())

        boxes = []
        labels = []
        ann_objects = annot["annotation"].get("object", [])
        if not isinstance(ann_objects, list):
            ann_objects = [ann_objects]

        for obj in ann_objects:
            bbox = obj["bndbox"]
            xmin, ymin, xmax, ymax = float(bbox["xmin"]), float(bbox["ymin"]), float(bbox["xmax"]), float(bbox["ymax"])
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(1)

        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        labels = torch.as_tensor(labels, dtype=torch.int64)
        target = {"boxes": boxes, "labels": labels, "image_id": torch.tensor(idx)}

        

        return image, target

    def __len__(self):
        return len(self.image_files)

In [ ]:

from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
import torch
import torch.nn as nn
from torchvision.models.detection.backbone_utils import BackboneWithFPN
from torchvision.models import resnet152, ResNet152_Weights

from torchvision.ops import misc as misc_nn_ops

def get_resnet152_fpn_with_dropout(dropout_rate=0.0):
    print('drop out rate = ', dropout_rate)
    backbone = resnet_fpn_backbone(
        'resnet152',
        weights=ResNet152_Weights.DEFAULT,
        trainable_layers=5,
        norm_layer=nn.BatchNorm2d  # or SyncBatchNorm if using distributed
        
    )
    # You could modify the returned backbone or model to insert dropout in heads
    return backbone


In [ ]:
from torchvision.ops import generalized_box_iou_loss
import torch.nn.functional as F

def custom_fastrcnn_loss(self, class_logits, box_regression, labels, regression_targets):
    # Standard classification loss
    classification_loss = F.cross_entropy(class_logits, labels)

    # Select positive samples
    sampled_pos_inds_subset = torch.where(labels > 0)[0]
    N = labels.numel()

    if sampled_pos_inds_subset.numel() == 0:
        box_loss = torch.tensor(0.0, device=labels.device)
    else:
        box_regression = box_regression[sampled_pos_inds_subset, labels[sampled_pos_inds_subset]]
        regression_targets = regression_targets[sampled_pos_inds_subset]

        box_loss = generalized_box_iou_loss(
            box_regression, regression_targets, reduction="sum"
        )

    box_loss = box_loss / N
    return classification_loss, box_loss


In [ ]:
def get_model(num_classes,dropout_rate=0.0):
    backbone = get_resnet152_fpn_with_dropout(dropout_rate=dropout_rate)

    anchor_generator = AnchorGenerator(
        sizes=((16,), (166,), (98,), (73,), (257,)),
        aspect_ratios=((0.89,), (1.54,), (0.53,), (1.0,), (1.04,))
    )

    model = FasterRCNN(
        backbone=backbone,
        num_classes=num_classes,
        rpn_anchor_generator=anchor_generator
    )

    # Replace only the fastrcnn_loss method
    from types import MethodType
    model.roi_heads.fastrcnn_loss = MethodType(custom_fastrcnn_loss, model.roi_heads)

    # Keep your transform normalization
    model.transform.image_mean = [0.0, 0.0, 0.0]
    model.transform.image_std = [1.0, 1.0, 1.0]

    return model


In [ ]:
import albumentations as A

def get_transform(train=False):
    if train:
        return A.Compose([
            A.HorizontalFlip(p=0.2),
                A.VerticalFlip(p=0.3),

            A.RandomBrightnessContrast(p=0.2),
A.Affine(
    translate_percent=0.05,     # Up to ±5% shift horizontally and vertically
    scale=(0.9, 1.1),           # Random scale between 90% and 110%
    rotate=(-10, 10),           # Random rotation between -10 and 10 degrees
    p=0.5
),            # A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.1, rotate_limit=10, p=0.5),

            ToTensorV2()
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))
    else:
        return A.Compose([
            ToTensorV2()
        ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['labels']))



In [ ]:
def train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10):
    model.train()
    running_loss = 0.0
    # for i, (images, targets) in enumerate(tqdm(data_loader ,desc=f"poch {epoch}",)):
    for i, (images, targets) in enumerate(tqdm(data_loader, desc=f"Epoch {epoch+1}/{num_epochs}", total=len(data_loader))):



        images = [torch.tensor(img, dtype=torch.float32).permute(2, 0, 1).to(device) for img in images]
        from torchvision.transforms.functional import normalize

        images = [normalize(img.float() / 255.0,
                    mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]) for img in images]
       


        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        # print("Boxes:", targets[0]["boxes"])
        # print("Labels:", targets[0]["labels"])


        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()

        # solution of nan
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        optimizer.step()
        # print(losses)
        running_loss += losses.item()
        if i % print_freq == 0:
            print(f"Iteration {i}: Loss: {losses.item():.4f}")
    print(running_loss)
    avg_loss = running_loss / len(data_loader)
    train_losses.append(avg_loss)

    print(f"Epoch {epoch} Average Loss: {avg_loss:.4f}")

score eval with patience

In [ ]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision
from torchvision.transforms.functional import normalize
import torch
from tqdm.notebook import tqdm

# Global variables for patience and mAP tracking
best_map = float('-inf')
epochs_without_improvement = 0
patience = 12
map_values = []  # <-- Store mAP values per epoch for plotting
avg_scores=[]
map_values_5=[]

@torch.no_grad()
def evaluate(model, data_loader, device, conf_threshold=0.2):
    global best_map, epochs_without_improvement, map_values

    model.eval()
    all_boxes = []
    all_labels = []
    all_scores = []

    metric = MeanAveragePrecision()

    for images, targets in tqdm(data_loader, desc="Evaluating"):
        images = [torch.tensor(img).float() / 255.0 for img in images]
        images = [normalize(img.permute(2, 0, 1), mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225]).to(device) for img in images]

        outputs = model(images)

        # Store filtered outputs for example score display
        for output in outputs:
            scores = output['scores']
            keep = scores > conf_threshold
            all_boxes.append(output['boxes'][keep].cpu())
            all_labels.append(output['labels'][keep].cpu())
            all_scores.append(scores[keep].cpu())



       
        # Format predictions
        preds = [{
            'boxes': out['boxes'].detach().cpu(),
            'scores': out['scores'].detach().cpu(),
            'labels': out['labels'].detach().cpu()
        } for out in outputs]

        # Format targets
        targets = [{
            'boxes': t['boxes'].clone().detach().cpu(),
            'labels': t['labels'].clone().detach().cpu()
        } for t in targets]

        metric.update(preds, targets)

    # Show example outputs
    print("Evaluation Complete.")
    for i in range(min(5, len(all_boxes))):
        print(f"Image {i}: Boxes: {all_boxes[i].numpy()}, Scores: {all_scores[i].numpy()}")

    # Compute mAP results
    result = metric.compute()
    current_map = result['map'].item()
    map_values.append(current_map)
    map_values_5.append(result['map_50'].item())


    print(f"\n📊 mAP: {current_map:.4f}")
    print(f"mAP@0.5: {result['map_50'].item():.4f}")
    print(f"mAP@0.5:0.95: {result['map'].item():.4f}")

    #########################
    all_scores_flattened = torch.cat(all_scores)
    average_score = all_scores_flattened.mean().item()
    print(f"\nAverage score across all boxes: {average_score:.4f}\n")
    avg_scores.append(average_score)
    #########################

    # Early stopping logic based on mAP
    if current_map > best_map + 1e-4:
        best_map = current_map
        epochs_without_improvement = 0
        print("✅ mAP improved.")
    else:
        epochs_without_improvement += 1
        print(f"⚠️ No mAP improvement. Count = {epochs_without_improvement}")

    if epochs_without_improvement >= patience:
        print("⛔ Early stopping triggered.")
        return True  # Stop training

    return False  # Continue training


In [ ]:
save_path = "/kaggle/working/metrics.json"
import json
import os

def save_metrics_json(filepath, train_losses, avg_scores, map_values, map_values_5):
    metrics = {
        "train_losses": train_losses,
        "avg_scores": avg_scores,
        "map_values": map_values,
        "map_values_5": map_values_5
    }
    with open(filepath, "w") as f:
        json.dump(metrics, f)


In [1]:

#for now training is kept off
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import ConcatDataset
import torch

batch_size = 1
num_epochs = 120
dropout_rate=.1

# Dataset 1
image_dir = '/kaggle/input/ship-dataset-2/Dataset/training/images'
annotation_dir = '/kaggle/input/ship-dataset-2/Dataset/training/annotations'
print(os.path.isdir(image_dir))
print(os.path.isdir(annotation_dir))

all_images1 = sorted([file for file in os.listdir(image_dir) if file.lower().endswith((".jpg", ".jpeg", ".png"))])
train_images1, val_images1 = train_test_split(all_images1, test_size=0.2, random_state=42)

train_ds1 = BoatDetectionDataset(image_dir, annotation_dir, image_list=train_images1, transform=get_transform(train=True))
val_ds1 = BoatDetectionDataset(image_dir, annotation_dir, image_list=val_images1)

# Dataset 2

train_image_dir = '/kaggle/input/dataset/train'
train_annotation_dir = '/kaggle/input/dataset/train'
val_image_dir = '/kaggle/input/dataset/valid'
val_annotation_dir = '/kaggle/input/dataset/valid'

print(os.path.isdir(train_image_dir))
print(os.path.isdir(val_image_dir))
print(os.path.isdir(train_annotation_dir))
print(os.path.isdir(val_annotation_dir))


train_images2 = sorted([file for file in os.listdir(train_image_dir) if file.lower().endswith((".jpg", ".jpeg", ".png"))])
val_images2 = sorted([file for file in os.listdir(val_image_dir) if file.lower().endswith((".jpg", ".jpeg", ".png"))])

train_ds2 = BoatDetectionDataset(train_image_dir, train_annotation_dir, image_list=train_images2, transform=get_transform(train=True))
val_ds2 = BoatDetectionDataset(val_image_dir, val_annotation_dir, image_list=val_images2)

# Combine the two datasets
train_ds = ConcatDataset([train_ds1, train_ds2])
val_ds = ConcatDataset([val_ds1, val_ds2])

# Dataloaders
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = torch.utils.data.DataLoader(val_ds, batch_size=batch_size, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


model = get_model(num_classes=2,dropout_rate=dropout_rate)


# To load:
# checkpoint = torch.load('mdsdo_GIoU_45.pth')
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])


model.to(device)
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.0005, momentum=0.9, weight_decay=0.0005)

epoch_counter = 0
train_losses=[]

for epoch in range(num_epochs):
    train_one_epoch(model, optimizer, train_loader, device, epoch, print_freq=1000)
    
    # Check early stopping inside evaluate
    should_stop = evaluate(model, val_loader, device)


    # Save checkpoint every 10 epochs


    # Early stopping condition
    if should_stop:
        print(f"Early stopping at epoch {epoch + 1}")
        break
        
    if epoch and (epoch) % 5 == 0:
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, f'/kaggle/working/mdsdo_GIoU_{epoch}.pth')


    save_metrics_json(save_path, train_losses, avg_scores, map_values, map_values_5)
    epoch_counter+=1

    


# Save final model (whether early stopped or completed)
print("Training and evaluation complete!")
torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, f'/kaggle/working/mdsdo_GIoU_{epoch_counter if epoch_counter<num_epochs else num_epochs-1}.pth')

False
False


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/ship-dataset-2/Dataset/training/images'

# #plot average loss

In [ ]:


import matplotlib.pyplot as plt

plt.plot(train_losses, label='loss over epoch')
plt.xlabel('EPOCH')
plt.ylabel('loss')
plt.title('epoch vs loss')
plt.legend()
plt.grid(True)
plt.show()

# #plot  scores

In [ ]:


import matplotlib.pyplot as plt

epochs = list(range(len(avg_scores)))

plt.figure(figsize=(10, 6))
plt.plot(epochs, avg_scores, label='Average Score', marker='o')
plt.plot(epochs, map_values, label='mAP@.5:.95', marker='s')
plt.plot(epochs, map_values_5, label='mAP@.5', marker='^')  # Valid marker

plt.xlabel("Epoch")
plt.ylabel("Scores")
plt.title("Detection Metrics During Training on validation data")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()



# Testing

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import numpy as np
from torchvision.transforms.functional import normalize
from PIL import Image, ImageDraw, ImageFont
import torch
from tqdm.notebook import tqdm

@torch.no_grad()
def test(model, data_loader, device, conf_threshold=0.2):
    model.eval()
    all_boxes = []
    all_labels = []
    all_scores = []

    for batch_idx, (images, targets) in enumerate(tqdm(data_loader, desc="Testing")):
        original_images = images  # Keep originals for visualization

        # Convert NumPy arrays to tensors and normalize
        images = [torch.tensor(img).float() / 255.0 for img in original_images]
        images = [normalize(img.permute(2, 0, 1), mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225]).to(device) for img in images]

        outputs = model(images)

        for i, output in enumerate(outputs):
            boxes = output['boxes'].cpu()
            scores = output['scores'].cpu()
            labels = output['labels'].cpu()

            keep = scores > conf_threshold
            boxes = boxes[keep]
            scores = scores[keep]
            labels = labels[keep]

            all_boxes.append(boxes)
            all_labels.append(labels)
            all_scores.append(scores)

            # Visualize predictions and ground truths
            img_np = original_images[i].copy().astype(np.uint8)
            img_rgb = cv2.cvtColor(img_np, cv2.COLOR_BGR2RGB)

            fig, ax = plt.subplots(1, figsize=(10, 6))
            ax.imshow(img_rgb)

            # Plot predicted boxes (green)
            for box, score in zip(boxes, scores):
                x1, y1, x2, y2 = map(int, box.tolist())
                rect = patches.Rectangle((x1, y1), x2 - x1, y2 - y1,
                                         linewidth=1, edgecolor='b', facecolor='none', label='Prediction')
                ax.add_patch(rect)
                ax.text(x1, y1 - 10, f"{score:.2f}", color='lime', fontsize=8, backgroundcolor='black')
                cv2.putText(img_np, f"boat {x1}, {y1}", (x1, y1 - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 1)


            # Plot target boxes (red)
            if targets is not None:
                target_boxes = targets[i]['boxes'].cpu()
                for tbox in target_boxes:
                    tx1, ty1, tx2, ty2 = map(int, tbox.tolist())
                    rect = patches.Rectangle((tx1, ty1), tx2 - tx1, ty2 - ty1,
                                             linewidth=1, edgecolor='r', facecolor='none', label='Target')
                    ax.add_patch(rect)

            # Avoid duplicate legend entries
            handles, labels = ax.get_legend_handles_labels()
            by_label = dict(zip(labels, handles))
            ax.legend(by_label.values(), by_label.keys())

            ax.set_title(f"Predicted vs Target boxes - Image {batch_idx * len(images) + i}")
            ax.axis("off")
            plt.show()

    print("Testing Complete.")
    if all_scores:
        all_scores_cat = torch.cat(all_scores)
        average_score = all_scores_cat.mean().item()
    else:
        average_score = 0.0

    print("Average score:", average_score)


# Test loader creator function

In [ ]:
def testloader_kaggle(dataset=-1):
    if dataset == 2:
        test_image_dir = '/kaggle/input/ship-dataset-2/Dataset/test/images'
        test_annotation_dir = '/kaggle/input/ship-dataset-2/Dataset/test/annotations'
        
        print(os.path.isdir(test_image_dir))
        print(os.path.isdir(test_annotation_dir))
        
        
        test_images =sorted([file for file in os.listdir(test_image_dir) if file.lower().endswith((".jpg", ".jpeg", ".png"))])
        test_ds = BoatDetectionDataset(test_image_dir, test_annotation_dir, image_list=test_images,)
        test_loader = torch.utils.data.DataLoader(test_ds, batch_size=1, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
        return test_loader

    elif dataset == 1:
        test_image_dir = '/kaggle/input/dataset/test'
        test_annotation_dir = '/kaggle/input/dataset/test'
        
        print(os.path.isdir(test_image_dir))
        print(os.path.isdir(test_annotation_dir))
                
        test_images =sorted([file for file in os.listdir(test_image_dir) if file.lower().endswith((".jpg", ".jpeg", ".png"))])
        test_ds = BoatDetectionDataset(test_image_dir, test_annotation_dir, image_list=test_images,)
        test_loader = torch.utils.data.DataLoader(test_ds, batch_size=1, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))
        return test_loader
        

    print('invalid operation')
    return None
        

# mAP Calculation



In [ ]:
from torchmetrics.detection.mean_ap import MeanAveragePrecision

def mAP_calculation(data_loader,model):
    
    metric = MeanAveragePrecision()
    for images, targets in data_loader:
        images = [torch.tensor(img).float() / 255.0 for img in images]
        images = [normalize(img.permute(2, 0, 1), mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225]).to(device) for img in images]
        model.eval()
        outputs = model(images)
    
        # Format prediction and target dictionaries
        preds = []
        for out in outputs:
            preds.append({
                'boxes': out['boxes'].detach().cpu(),
                'scores': out['scores'].detach().cpu(),
                'labels': out['labels'].detach().cpu(),
            })
    
        targets = [{  # <- you must have this
            'boxes': ann['boxes'].clone().detach(),
    'labels': ann['labels'].clone().detach()
    
    
        } for ann in targets]
    
        metric.update(preds, targets)
    
    result = metric.compute()
    print("mAP:", result["map"].item())
    print("mAP@0.5:", result["map_50"].item())
    print("mAP@0.5:0.95:", result["map"].item())

# load saved fucnction

In [ ]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = get_model(2)
# model.to(device)
# checkpoint = torch.load('mdsdo_GIoU_45.pth')
# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# mAP for dataset1

In [ ]:

test_loader = testloader_kaggle(dataset=1)
mAP_calculation(test_loader,model)


# mAP For Dataset2

In [ ]:


test_loader = testloader_kaggle(dataset=2)

mAP_calculation(test_loader,model)


# overall mAP values on series of model state on test data

In [ ]:
from torch.utils.data import ConcatDataset
from torchmetrics.detection.mean_ap import MeanAveragePrecision

############################################### KAGGLE  ############################################################


#dataset1
test_image_dir = '/kaggle/input/dataset/test'
test_annotation_dir = '/kaggle/input/dataset/test'
print(os.path.isdir(test_image_dir))
print(os.path.isdir(test_annotation_dir))
test_images =sorted([file for file in os.listdir(test_image_dir) if file.lower().endswith((".jpg", ".jpeg", ".png"))])
test_ds1 = BoatDetectionDataset(test_image_dir, test_annotation_dir, image_list=test_images,)

#dataset2
test_image_dir = '/kaggle/input/ship-dataset-2/Dataset/test/images'
test_annotation_dir = '/kaggle/input/ship-dataset-2/Dataset/test/annotations'
print(os.path.isdir(test_image_dir))
print(os.path.isdir(test_annotation_dir))
test_images =sorted([file for file in os.listdir(test_image_dir) if file.lower().endswith((".jpg", ".jpeg", ".png"))])

test_ds2 = BoatDetectionDataset(test_image_dir, test_annotation_dir, image_list=test_images,)
test_ds = ConcatDataset([test_ds1, test_ds2])

test_loader = torch.utils.data.DataLoader(test_ds, batch_size=1, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))


####################################################################################################################

###############################################    COLAB    ########################################################

# #dataset1
# test_image_dir = '/content/drive/MyDrive/GEDS/test'
# test_annotation_dir = '/content/drive/MyDrive/GEDS/test'
# print(os.path.isdir(test_image_dir))
# print(os.path.isdir(test_annotation_dir))
# test_images =sorted([file for file in os.listdir(test_image_dir) if file.lower().endswith((".jpg", ".jpeg", ".png"))])
# test_ds1 = BoatDetectionDataset(test_image_dir, test_annotation_dir, image_list=test_images,)

# #dataset2
# test_image_dir = '/content/drive/MyDrive/Dataset/test/images'
# test_annotation_dir = '/content/drive/MyDrive/Dataset/test/annotations'
# print(os.path.isdir(test_image_dir))
# print(os.path.isdir(test_annotation_dir))
# test_images =sorted([file for file in os.listdir(test_image_dir) if file.lower().endswith((".jpg", ".jpeg", ".png"))])

# test_ds2 = BoatDetectionDataset(test_image_dir, test_annotation_dir, image_list=test_images,)
# test_ds = ConcatDataset([test_ds1, test_ds2])

# test_loader = torch.utils.data.DataLoader(test_ds, batch_size=1, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

########################################################################################################################
ind=0
test_map_values=[]
test_map_values_50=[]

# mpath=f'/kaggle/input/shipdetection_fasterrcnn_resnet152_accuracy77/pytorch/default/1/mdsdo_GIoU_0.pth'
# print(os.path.isfile(mpath))
pbar = tqdm()
while os.path.isfile(f'/kaggle/working/mdsdo_GIoU_{ind}.pth'):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    
    model = get_model(2,dropout_rate = dropout_rate)
    model.to(device)

    checkpoint = torch.load(f'/kaggle/working/mdsdo_GIoU_{ind}.pth',map_location=torch.device(device))
    
    #to run without CUDA
    
    # checkpoint = torch.load(f'/kaggle/input/shipdetection_fasterrcnn_resnet152_accuracy77/pytorch/default/1/mdsdo_GIoU_{ind}.pth',map_location=torch.device('cpu'))
    
    model.load_state_dict(checkpoint['model_state_dict'])
    # optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    model.eval()
    
    metric = MeanAveragePrecision()
    for images, targets in tqdm(test_loader, total=len(test_loader)):
        images = [torch.tensor(img).float() / 255.0 for img in images]
        images = [normalize(img.permute(2, 0, 1), mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225]).to(device) for img in images]
        outputs = model(images)
    
        # Format prediction and target dictionaries
        preds = []
        for out in outputs:
            preds.append({
                'boxes': out['boxes'].detach().cpu(),
                'scores': out['scores'].detach().cpu(),
                'labels': out['labels'].detach().cpu(),
            })
    
        targets = [{  # <- you must have this
            'boxes': ann['boxes'].clone().detach(),
    'labels': ann['labels'].clone().detach()
    
        } for ann in targets]
    
        metric.update(preds, targets)
    
    result = metric.compute()
    print("mAP:", result["map"].item())
    print("mAP@0.5:", result["map_50"].item())
    print("mAP@0.5:0.95:", result["map"].item())
    test_map_values.append(result["map"].item())
    test_map_values_50.append(result["map_50"].item())
    pbar.update(1)
    ind+=5
pbar.close()   

In [ ]:
# a=[3,2,87,4,9]
# b=list(range(0,(len(a)-1)*5+1,5))
# print(a)
# print(b)

In [ ]:
import matplotlib.pyplot as plt

# Sample data (replace with actual values)
epochs = list(range(0,(len(test_map_values)-1)*5+1,5))

# 1️⃣ Plot Figure: Line plot of metrics
plt.rcParams.update({'font.size': 14})
fig1, ax1 = plt.subplots(figsize=(12, 6))
ax1.plot(epochs, test_map_values, label='mAP@.5:.95', marker='s')
ax1.plot(epochs, test_map_values_50, label='mAP@.5', marker='^')
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Scores")
ax1.set_title("Detection Metrics During Training")
ax1.legend()
ax1.grid(True)
fig1.tight_layout()
fig1.savefig("detection_metrics_plot.png", dpi=300)

# 2️⃣ Table Figure: Show metrics in a standalone table
fig2, ax2 = plt.subplots(figsize=(10, 6))
ax2.axis('off')  # Hide axes

# Prepare table data
table_data = [[str(e), f"{m1:.2f}", f"{m2:.2f}"]
              for e, m1, m2 in zip(epochs, test_map_values, test_map_values_50)]

# Draw the table in the center
table = ax2.table(
    cellText=table_data,
    colLabels=["Epoch", "mAP@.5:.95", "mAP@.5"],
    loc='center',
    cellLoc='center',
    colLoc='center',
    bbox=[0, 0, 1, 1]  # Fill full area of the figure
)

table.auto_set_font_size(False)
table.set_fontsize(14)

# Adjust row height
for key, cell in table.get_celld().items():
    cell.set_height(0.06)  # Better vertical spacing for 14pt font

fig2.tight_layout()
fig2.savefig("detection_metrics_table.png", dpi=300)
plt.show()


# show imges with predicted Bounding Boxes

In [ ]:

test_loader = testloader_kaggle(dataset=1) #customize as per need dataset =1,2
pbar= tqdm(total= len(test_loader))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
test(
    model,
   test_loader,
    device,
    conf_threshold=0.4 # You can increase this to reduce false positives
)


# overall dataset confushion matrix 

In [ ]:
import numpy as np
import torch
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.utils.multiclass import unique_labels
from torchvision.ops import box_iou
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import ConcatDataset



############################################### KAGGLE  ############################################################


#dataset1
test_image_dir = '/kaggle/input/dataset/test'
test_annotation_dir = '/kaggle/input/dataset/test'
print(os.path.isdir(test_image_dir))
print(os.path.isdir(test_annotation_dir))
test_images =sorted([file for file in os.listdir(test_image_dir) if file.lower().endswith((".jpg", ".jpeg", ".png"))])
test_ds1 = BoatDetectionDataset(test_image_dir, test_annotation_dir, image_list=test_images,)

#dataset2
test_image_dir = '/kaggle/input/ship-dataset-2/Dataset/test/images'
test_annotation_dir = '/kaggle/input/ship-dataset-2/Dataset/test/annotations'
print(os.path.isdir(test_image_dir))
print(os.path.isdir(test_annotation_dir))
test_images =sorted([file for file in os.listdir(test_image_dir) if file.lower().endswith((".jpg", ".jpeg", ".png"))])

test_ds2 = BoatDetectionDataset(test_image_dir, test_annotation_dir, image_list=test_images,)
test_ds = ConcatDataset([test_ds1, test_ds2])

test_loader = torch.utils.data.DataLoader(test_ds, batch_size=1, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))


####################################################################################################################

###############################################    COLAB    ########################################################

# #dataset1
# test_image_dir = '/content/drive/MyDrive/GEDS/test'
# test_annotation_dir = '/content/drive/MyDrive/GEDS/test'
# print(os.path.isdir(test_image_dir))
# print(os.path.isdir(test_annotation_dir))
# test_images =sorted([file for file in os.listdir(test_image_dir) if file.lower().endswith((".jpg", ".jpeg", ".png"))])
# test_ds1 = BoatDetectionDataset(test_image_dir, test_annotation_dir, image_list=test_images,)

# #dataset2
# test_image_dir = '/content/drive/MyDrive/Dataset/test/images'
# test_annotation_dir = '/content/drive/MyDrive/Dataset/test/annotations'
# print(os.path.isdir(test_image_dir))
# print(os.path.isdir(test_annotation_dir))
# test_images =sorted([file for file in os.listdir(test_image_dir) if file.lower().endswith((".jpg", ".jpeg", ".png"))])

# test_ds2 = BoatDetectionDataset(test_image_dir, test_annotation_dir, image_list=test_images,)
# test_ds = ConcatDataset([test_ds1, test_ds2])

# test_loader = torch.utils.data.DataLoader(test_ds, batch_size=1, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

########################################################################################################################


# Class label mapping
class_names = ["background", "vessel"]
class_ids = [0, 1]

# Plot setup
fig, axes = plt.subplots(5, 2, figsize=(16, 20))
axes = axes.flatten()

iou_threshold = 0.5
score_thresholds = [round(t, 1) for t in np.arange(0.0, 1.0, 0.1)]

for idx, score_thresh in tqdm(enumerate(score_thresholds), total=len(score_thresholds)):
    all_preds = []
    all_trues = []

    for images, targets in test_loader:
        images = [torch.tensor(img).float() / 255.0 for img in images]
        images = [normalize(img.permute(2, 0, 1), mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225]).to(device) for img in images]

        outputs = model(images)

        for pred, target in zip(outputs, targets):
            pred_boxes = pred['boxes'].detach().cpu()
            pred_labels = pred['labels'].detach().cpu()
            pred_scores = pred['scores'].detach().cpu()

            true_boxes = target['boxes'].detach().cpu()
            true_labels = target['labels'].detach().cpu()

            # Apply score threshold
            keep = pred_scores >= score_thresh
            pred_boxes = pred_boxes[keep]
            pred_labels = pred_labels[keep]

            matched_gt = set()

            if len(pred_boxes) > 0 and len(true_boxes) > 0:
                ious = box_iou(pred_boxes, true_boxes)

                for pred_idx in range(len(pred_boxes)):
                    max_iou, gt_idx = ious[pred_idx].max(0)
                    if max_iou >= iou_threshold and gt_idx.item() not in matched_gt:
                        all_preds.append(1)  # vessel
                        all_trues.append(1)
                        matched_gt.add(gt_idx.item())
                    else:
                        all_preds.append(1)  # vessel
                        all_trues.append(0)  # background (FP)

            # False negatives: missed vessels
            for gt_idx in range(len(true_boxes)):
                if gt_idx not in matched_gt:
                    all_preds.append(0)  # background
                    all_trues.append(1)  # vessel

    # Compute confusion matrix
    cm = confusion_matrix(all_trues, all_preds, labels=class_ids)

    # Safe metric computation
    present_labels = set(unique_labels(all_trues, all_preds))
    if 1 in present_labels and 0 in present_labels:
        precision = precision_score(all_trues, all_preds, average='binary', pos_label=1, zero_division=0)
        recall = recall_score(all_trues, all_preds, average='binary', pos_label=1, zero_division=0)
        f1 = f1_score(all_trues, all_preds, average='binary', pos_label=1, zero_division=0)
    else:
        precision = precision_score(all_trues, all_preds, average='macro', zero_division=0)
        recall = recall_score(all_trues, all_preds, average='macro', zero_division=0)
        f1 = f1_score(all_trues, all_preds, average='macro', zero_division=0)

    # Plot
    ax = axes[idx]
    sns.heatmap(cm, annot=True, fmt='d',
                xticklabels=class_names, yticklabels=class_names,
                cmap='Blues', ax=ax)
    ax.set_xlabel('Predicted')
    ax.set_ylabel('True')
    ax.set_title(f"Threshold: {score_thresh} | P: {precision:.2f} R: {recall:.2f} F1: {f1:.2f}")

plt.tight_layout()
#save colab
# plt.savefig("/content/drive/MyDrive/confusion_mat.png", dpi=300, bbox_inches='tight')

#save kaggle
plt.savefig("/kaggle/working/confusion_mat.png", dpi=300, bbox_inches='tight')

plt.show()


# plot from saved files


In [ ]:
import matplotlib.pyplot as plt

# Load metrics
with open("/kaggle/working/metrics.json", "r") as f:
    data = json.load(f)

# Plot
if os.path.isfile ("/kaggle/working/metrics.json"):
    plt.figure(figsize=(10, 6))
    plt.plot(data["train_losses"], label="Train Loss")
    plt.plot(data["avg_scores"], label="Avg Score")
    plt.plot(data["map_values"], label="mAP")
    plt.plot(data["map_values_5"], label="mAP@0.5")
    plt.xlabel("Epoch")
    plt.ylabel("Value")
    plt.title("Training Metrics Over Epochs")
    plt.legend()
    plt.grid(True)
    plt.show()
